In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset3') 

# Actions that we try to detect
actions = np.array(['black','white','red'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'black': 0, 'white': 1, 'red': 2}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(90, 30, 1662)

In [14]:
np.array(labels).shape

(90,)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(90, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
y_test.shape

(5, 3)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 835ms/step - loss: 2.4393 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 0s 60ms/step - loss: 4.6929 - categorical_accuracy: 0.4588
Epoch 3/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.8160 - categorical_accuracy: 0.3529
Epoch 4/2000
3/3 [==============================] - 0s 117ms/step - loss: 2.9215 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 0s 116ms/step - loss: 2.4787 - categorical_accuracy: 0.4118
Epoch 6/2000
3/3 [==============================] - 0s 48ms/step - loss: 1.0701 - categorical_accuracy: 0.2941
Epoch 7/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.0245 - categorical_accuracy: 0.3412
Epoch 8/2000
3/3 [==============================] - 1s 220ms/step - loss: 0.9694 - categorical_accuracy: 0.3176
Epoch 9/2000
3/3 [==============================] - 1s 109ms/step - loss: 0.8795 - categorical_accuracy: 0.5

3/3 [==============================] - 0s 100ms/step - loss: 1.4889e-05 - categorical_accuracy: 1.0000
Epoch 143/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.4543e-05 - categorical_accuracy: 1.0000
Epoch 144/2000
3/3 [==============================] - 0s 90ms/step - loss: 1.4376e-05 - categorical_accuracy: 1.0000
Epoch 145/2000
3/3 [==============================] - 0s 109ms/step - loss: 1.4191e-05 - categorical_accuracy: 1.0000
Epoch 146/2000
3/3 [==============================] - 0s 75ms/step - loss: 1.3958e-05 - categorical_accuracy: 1.0000
Epoch 147/2000
3/3 [==============================] - 0s 77ms/step - loss: 1.3640e-05 - categorical_accuracy: 1.0000
Epoch 148/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.3509e-05 - categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.3331e-05 - categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 63ms/step - loss: 

3/3 [==============================] - 0s 142ms/step - loss: 3.2367e-06 - categorical_accuracy: 1.0000
Epoch 281/2000
3/3 [==============================] - 0s 108ms/step - loss: 3.2072e-06 - categorical_accuracy: 1.0000
Epoch 282/2000
3/3 [==============================] - 1s 383ms/step - loss: 3.1568e-06 - categorical_accuracy: 1.0000
Epoch 283/2000
3/3 [==============================] - 0s 152ms/step - loss: 3.1525e-06 - categorical_accuracy: 1.0000
Epoch 284/2000
3/3 [==============================] - 1s 291ms/step - loss: 3.1105e-06 - categorical_accuracy: 1.0000
Epoch 285/2000
3/3 [==============================] - 1s 170ms/step - loss: 3.0908e-06 - categorical_accuracy: 1.0000
Epoch 286/2000
3/3 [==============================] - 0s 123ms/step - loss: 3.0880e-06 - categorical_accuracy: 1.0000
Epoch 287/2000
3/3 [==============================] - 0s 89ms/step - loss: 3.0530e-06 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 1s 311ms/step - lo

3/3 [==============================] - 0s 147ms/step - loss: 1.3393e-06 - categorical_accuracy: 1.0000
Epoch 419/2000
3/3 [==============================] - 1s 488ms/step - loss: 1.3337e-06 - categorical_accuracy: 1.0000
Epoch 420/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.3183e-06 - categorical_accuracy: 1.0000
Epoch 421/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.3085e-06 - categorical_accuracy: 1.0000
Epoch 422/2000
3/3 [==============================] - 0s 123ms/step - loss: 1.2972e-06 - categorical_accuracy: 1.0000
Epoch 423/2000
3/3 [==============================] - 0s 77ms/step - loss: 1.2986e-06 - categorical_accuracy: 1.0000
Epoch 424/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.2916e-06 - categorical_accuracy: 1.0000
Epoch 425/2000
3/3 [==============================] - 1s 360ms/step - loss: 1.2762e-06 - categorical_accuracy: 1.0000
Epoch 426/2000
3/3 [==============================] - 1s 73ms/step - los

3/3 [==============================] - 1s 652ms/step - loss: 7.0963e-07 - categorical_accuracy: 1.0000
Epoch 556/2000
3/3 [==============================] - 0s 109ms/step - loss: 7.0402e-07 - categorical_accuracy: 1.0000
Epoch 557/2000
3/3 [==============================] - 0s 109ms/step - loss: 7.0262e-07 - categorical_accuracy: 1.0000
Epoch 558/2000
3/3 [==============================] - 1s 390ms/step - loss: 6.9982e-07 - categorical_accuracy: 1.0000
Epoch 559/2000
3/3 [==============================] - 1s 145ms/step - loss: 6.9701e-07 - categorical_accuracy: 1.0000
Epoch 560/2000
3/3 [==============================] - 0s 94ms/step - loss: 6.9280e-07 - categorical_accuracy: 1.0000
Epoch 561/2000
3/3 [==============================] - 1s 319ms/step - loss: 6.9000e-07 - categorical_accuracy: 1.0000
Epoch 562/2000
3/3 [==============================] - 2s 369ms/step - loss: 6.8579e-07 - categorical_accuracy: 1.0000
Epoch 563/2000
3/3 [==============================] - 0s 75ms/step - los

...

3/3 [==============================] - 0s 147ms/step - loss: 4.3476e-08 - categorical_accuracy: 1.0000
Epoch 1381/2000
3/3 [==============================] - 1s 256ms/step - loss: 4.3476e-08 - categorical_accuracy: 1.0000
Epoch 1382/2000
3/3 [==============================] - 1s 163ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1383/2000
3/3 [==============================] - 0s 61ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1384/2000
3/3 [==============================] - 0s 98ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1385/2000
3/3 [==============================] - 0s 107ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1386/2000
3/3 [==============================] - 1s 300ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1387/2000
3/3 [==============================] - 0s 171ms/step - loss: 4.4879e-08 - categorical_accuracy: 1.0000
Epoch 1388/2000
3/3 [==============================] - 0s 59ms/ste

3/3 [==============================] - 0s 147ms/step - loss: 2.9452e-08 - categorical_accuracy: 1.0000
Epoch 1518/2000
3/3 [==============================] - 0s 95ms/step - loss: 2.9452e-08 - categorical_accuracy: 1.0000
Epoch 1519/2000
3/3 [==============================] - 1s 526ms/step - loss: 2.9452e-08 - categorical_accuracy: 1.0000
Epoch 1520/2000
3/3 [==============================] - 0s 122ms/step - loss: 2.9452e-08 - categorical_accuracy: 1.0000
Epoch 1521/2000
3/3 [==============================] - 0s 127ms/step - loss: 2.8049e-08 - categorical_accuracy: 1.0000
Epoch 1522/2000
3/3 [==============================] - 1s 251ms/step - loss: 2.8049e-08 - categorical_accuracy: 1.0000
Epoch 1523/2000
3/3 [==============================] - 0s 85ms/step - loss: 2.8049e-08 - categorical_accuracy: 1.0000
Epoch 1524/2000
3/3 [==============================] - 1s 335ms/step - loss: 2.9452e-08 - categorical_accuracy: 1.0000
Epoch 1525/2000
3/3 [==============================] - 1s 109ms/st

3/3 [==============================] - 0s 105ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1654/2000
3/3 [==============================] - 0s 141ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1655/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1656/2000
3/3 [==============================] - 0s 82ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000 0s - loss: 2.6077e-08 - categorical_accuracy: 1.
Epoch 1657/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1658/2000
3/3 [==============================] - 1s 355ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1659/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.1037e-08 - categorical_accuracy: 1.0000
Epoch 1660/2000
3/3 [==============================] - 0s 131ms/step - loss: 1.9634e-08 - categorical_accuracy: 1.0000
Epoch 1661/2000
3/3

3/3 [==============================] - 0s 174ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1790/2000
3/3 [==============================] - 1s 305ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1791/2000
3/3 [==============================] - 1s 166ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1792/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1793/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1794/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1795/2000
3/3 [==============================] - 1s 400ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1796/2000
3/3 [==============================] - 0s 137ms/step - loss: 1.4025e-08 - categorical_accuracy: 1.0000
Epoch 1797/2000
3/3 [==============================] - 0s 92ms/st

3/3 [==============================] - 0s 110ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1928/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1929/2000
3/3 [==============================] - 2s 543ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1930/2000
3/3 [==============================] - 0s 54ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1931/2000
3/3 [==============================] - 0s 80ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1932/2000
3/3 [==============================] - 0s 128ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1933/2000
3/3 [==============================] - 1s 80ms/step - loss: 1.1220e-08 - categorical_accuracy: 1.0000
Epoch 1934/2000
3/3 [==============================] - 0s 194ms/step - loss: 9.8172e-09 - categorical_accuracy: 1.0000
Epoch 1935/2000
3/3 [==============================] - 0s 104ms/ste

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [26]:
res = model.predict(X_test)

In [27]:
actions[np.argmax(res[4])]

'black'

In [28]:
actions[np.argmax(y_test[4])]

'black'

In [29]:
model.save('test3.h5')

In [11]:
model.load_weights('test3.h5')

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:

yhat = model.predict(X_test)

In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
red
<class 'mediapipe.python.solution_base.SolutionOutputs'>
red
<class 'mediapipe.python.solution_base.SolutionOutputs'>
red
<class 'mediapipe.python.solution_base.SolutionOutputs'>
red
<class 'mediapipe.python.solution_base.SolutionOutputs'>


white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOutputs'>
white
<class 'mediapipe.python.solution_base.SolutionOu